In [1]:
import numpy as np
import pandas as pd
from landmarks import holostic_model
from preprocess import load_data,get_interesting_idx
import matplotlib.pyplot as plt
from tensorflow.keras import models
import tensorflow as tf
import pickle
import os
import cv2
import mediapipe as mp

In [9]:
with open("Label_encoder_19.pkl",'rb') as file:
    label_encode = pickle.load(file)

In [3]:
model = models.load_model("20_action_v5_acc_98_valacc_86")

In [4]:
mp_model = holostic_model()
idx = get_interesting_idx()
data_load = load_data(idx)

In [5]:
# signs = os.listdir(os.path.join("Real example signs"))

In [6]:
# for sign in signs:
#     value = mp_model.extract_values(os.path.join("Real example signs",sign))
#     value = data_load.load_no_sign_data(value)
#     pred = model.predict(tf.expand_dims(value,0))
#     confidence = round((pred[0][np.argmax(pred.squeeze())])*100,2)
#     pred = label_encode.inverse_transform([[np.argmax(pred.squeeze())]])
#     print(f"Acttual sign : {sign} Predicted sign : {pred} confidence : {confidence} %")

1/1 [==============================] - 0s 262ms/step
Acttual sign : After.mp4 Predicted sign : [['thankyou']] confidence : 52.95 %
1/1 [==============================] - 0s 33ms/step
Acttual sign : All.mp4 Predicted sign : [['happy']] confidence : 87.31 %
1/1 [==============================] - 0s 31ms/step
Acttual sign : Apple.mp4 Predicted sign : [['dad']] confidence : 91.49 %
1/1 [==============================] - 0s 32ms/step
Acttual sign : Bad.mp4 Predicted sign : [['thankyou']] confidence : 97.67 %
1/1 [==============================] - 0s 31ms/step
Acttual sign : Car.mp4 Predicted sign : [['sleepy']] confidence : 99.92 %
1/1 [==============================] - 0s 32ms/step
Acttual sign : Dad.mp4 Predicted sign : [['dad']] confidence : 100.0 %
1/1 [==============================] - 0s 30ms/step
Acttual sign : Face.mp4 Predicted sign : [['face']] confidence : 45.54 %
1/1 [==============================] - 0s 32ms/step
Acttual sign : Go.mp4 Predicted sign : [['go']] confidence : 73.4

# Real time model prediction development

In [5]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

def detection_model(img,model):
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable=False
        result=model.process(img)
        img.flags.writeable=True
        img=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        return img,result
    
def draw_style(img,result):
    #face landmark
    mp_drawing.draw_landmarks(img, result.face_landmarks,mp_holistic.FACEMESH_CONTOURS,
                                    mp_drawing.DrawingSpec(color=(220, 103, 112),thickness=1,circle_radius=1),
                                    mp_drawing.DrawingSpec(color=(149, 234, 149),thickness=2,circle_radius=1))

    #pose landmark
    mp_drawing.draw_landmarks(img, result.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(20, 19, 247),thickness=2,circle_radius=2),
                            mp_drawing.DrawingSpec(color=(149, 234, 149),thickness=2,circle_radius=1))
    #left hand landmark
    mp_drawing.draw_landmarks(img, result.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(169, 42, 27),thickness=2,circle_radius=2),
                            mp_drawing.DrawingSpec(color=(254, 114, 236),thickness=2,circle_radius=1))
    #Right habd landmark
    mp_drawing.draw_landmarks(img, result.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(254, 114, 236),thickness=2,circle_radius=2),
                            mp_drawing.DrawingSpec(color=(169, 42, 27),thickness=2,circle_radius=1))
    

def key_values(results):
        pose = np.array([[result.x,result.y,result.z] for result in results.pose_landmarks.landmark]) if results.pose_landmarks else np.zeros((33,3)) 
        face = np.array([[result.x,result.y,result.z] for result in results.face_landmarks.landmark]) if results.face_landmarks else np.zeros((468,3))
        right_hand = np.array([[result.x,result.y,result.y] for  result in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else np.zeros((21,3))
        left_hand = np.array([[result.x,result.y,result.z] for result in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.zeros((21,3))
        return np.concatenate([face,left_hand,pose,right_hand])

In [6]:
# Definig a container to hold the frames for prediction
seq = []
# Defining variable to hold the predicted signs
sign =[]

hand = []

cap=cv2.VideoCapture(0)

#calling mediapipe line model and as to holistic variable
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.7) as holistic:

#capturing frames and looping to show as video
    while cap.isOpened():
        ret, frame=cap.read() # ret will be bool (when runing it will be true)
        # to strat while loop and below else is to break the loop
        if ret == True:
            #Converting to rgb and unwriteable to better with mediapipe and reversing and output image and result of model
            img,result=detection_model(frame,holistic)

            #drawing and extracting the result in the image 
            draw_style(img,result)
            seq.append(key_values(result))

            # to mirror the output image parameter 1 is use to flip along the y axix.
            image = cv2.flip(frame,1)

            # logic to check hand is detected in previous frames
            if result.right_hand_landmarks or result.left_hand_landmarks:
                hand.append(1)
            else:
                hand.append(0)
            
            # making the prediction for every 30 frames
            if len(seq)%40 == 0 and sum(hand[-40:]) >10 :
                value = np.array(seq[-30:]).squeeze()
                value = data_load.load_no_sign_data(value)
                pred = model.predict(tf.expand_dims(value,0))
                confidence = round((pred[0][np.argmax(pred.squeeze())])*100,2)
                pred = label_encode.inverse_transform([[np.argmax(pred.squeeze())]])
                sign.append(pred[0][0])
                seq = []
            try:
                if confidence > 95 and len(sign) >0:
                    cv2.putText(image,' '.join(sign),(350,150),cv2.FONT_HERSHEY_TRIPLEX,1,(41, 172, 252),2)
                    confidence = 0
            except:
                pass

            try:
                if confidence < 95 and len(sign) >0:
                    cv2.putText(image,' '.join(sign[-3:]),(50,50),cv2.FONT_HERSHEY_TRIPLEX,1,(41, 172, 252),2)
                    confidence = 0
            except:
                pass


            # displaying the resluted image
            cv2.imshow('Capturing',image)

            if cv2.waitKey(25) & 0xff == ord("q"):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 40ms/step
